In [167]:
import requests
from bs4 import BeautifulSoup
import string
import time
import re
import string
import os
from collections import Counter,OrderedDict
import nltk
from nltk.tokenize import word_tokenize
import glob
import pickle
import math
import operator

In [168]:
unigram_termcount = {}
unigram_corpus_count = 0
path = 'corpus'
smoothening_factor = 0.35
number_of_docs = len(glob.glob('corpus/*.txt'))
with open ('unigram_invertedlist_count.pkl', 'rb') as f: 
        unigram_invertedlist_count = pickle.load(f)
with open('inverted_index.pkl', 'rb') as f:
        inverted_index = pickle.load(f)

In [169]:
for filename in glob.glob("corpus/*.txt"):
    fo = open(filename, "r")
    data = fo.read()
    tokens = nltk.word_tokenize(data)
    unigramlist = nltk.word_tokenize(data)
    unigram_termcount[os.path.basename(filename).split(".")[0]] = len(unigramlist)
    unigram_corpus_count = unigram_corpus_count + len(unigramlist)
    unigram_dict = Counter(unigramlist)
    fo.close()

In [170]:

def casefolding(data):
    return data.lower()

def punctuationHandling(data):
    regex = r"(?<!\d)[.,;:*!\"\'#$%&()+/<=>?@[\]_^`~{|}∑α](?!\d)"
    data = re.sub(regex, "", data, 0)
    regex = r"[;:*!\"\'#$%&()+/<=>?@[\]_^`~{|}]"
    data = re.sub(regex, "", data, 0)
#     data = data.strip(string.punctuation)
    return data

def removeWhitespace(data):
    data = ' '.join(data.split())
    return data

def getDocWordFreq(word,document):
#     print(unigram_invertedlist_count.get(word))
    value = 0
    if unigram_invertedlist_count.get(word):
        for val in unigram_invertedlist_count[word][1]:
            if(val[0]==document):
                value = val[1]
    return value
def getCorpusWordFreq(word):
    if unigram_invertedlist_count.get(word):
        return unigram_invertedlist_count[word][0]
    else:
        return 0
def query_preprocessor(filepath = 'test-collection/cacm.query.txt'):
    with open(filepath) as f: queries = f.read()
    queries = [l.replace('</DOCNO>', '').replace('\n', ' ').replace('</DOC>', '').replace('<DOC>', '')[1:] for l in queries.split('<DOCNO>')]
    queries = [re.sub(r'^\d*\s\s', '',l) for l in queries]
    queries = [s.lower() for s in queries]
    queries = [punctuationHandling(query) for query in queries]
    queries = [removeWhitespace(query) for query in queries]
    #with open(filepath, 'w') as f: f.write('\n'.join(queries))
    return queries
all_queries = query_preprocessor()[1:]

In [171]:
def writeToFile(queryid,queryname,lmscore_dict):
    fo = open("task1-bm25/"+ "Q" + str(queryid) +".txt", "w")
    for key,val in lmscore_dict.items():
        rank = list(lmscore_dict.keys()).index(key)+1
#         print(queryid,"\tQ0\t",key,"\t",rank,"\t",val,"LMDirichlet\n")
        fo.write(str(queryid)+"\tQ0\t"+str(key)+"\t"+str(rank)+"\t"+str(val)+"\t ccisneu_wordunigram_BM25\n")
    fo.close()

In [172]:
def get_bm25(document,query,document_bm25_score_dict):
    total_score = 0
    score = 0
    query_list = query.split(" ")
    
    average_length = unigram_corpus_count / number_of_docs
    for query_word in query_list: 
        R = 0.0
        r = 0.0
        #number of docs containing the term
        n = unigram_invertedlist_count[query_word][0]
        
        #Total number of documents
        N = number_of_docs
        k1 = 1.2
        k2 = 100
        #freq of the word in query
        qf = query_list.count(query_word)
        
        #freq of word in the doc
        f = getDocWordFreq(query_word,document)
        
        #to be calculated using b= 0.75
        b = 0.75
        
        K = k1*((1-b) + b*(float(unigram_termcount[document])/float(average_length)))
        
        smoothening = 0.5
        
        first_part = math.log(((r + 0.5)/(R - r + 0.5))/((n - r + 0.5)/(N - n - R + r + 0.5)))
        second_part = ((k1 + 1)*f)/(K + f)
        third_part = ((k2 + 1) * qf)/(k2 + qf)
        score = first_part*second_part*third_part
        
        total_score += score
    document_bm25_score_dict[document] = total_score
    return document_bm25_score_dict

In [173]:
def populate_bm25(queryString):
    document_bm25_score_dict = {}
    docList = []
    query_list = queryString.split(" ")
    
    for query in query_list:
        if unigram_invertedlist_count.get(query):
            query_doclist = unigram_invertedlist_count[query][1]
            for i in query_doclist:
                docList.append(i[0])
        else:
            query_doclist = []
            
    for docid in docList:
        document_bm25_score_dict = get_bm25(docid,queryString,document_bm25_score_dict)
        document_bm25_score_dict = dict(sorted(document_bm25_score_dict.items(), key=operator.itemgetter(1), reverse=True)[:100])
    return document_bm25_score_dict

In [175]:
for i in range(len(all_queries)):
    bm25_score_dict = populate_bm25(all_queries[i])
    print(bm25_score_dict)
#     writeToFile(i+1,all_queries[i],bm25_score_dict)

{'CACM-2319': 25.208089861457385, 'CACM-2379': 22.11436354208018, 'CACM-1519': 21.658086017113753, 'CACM-1523': 20.438693585535642, 'CACM-1410': 20.272566903127615, 'CACM-1605': 20.25279110973632, 'CACM-2380': 20.17275445307061, 'CACM-1591': 19.671371155575276, 'CACM-1033': 19.66867997903659, 'CACM-1698': 19.14661946708638, 'CACM-1749': 19.113926855448092, 'CACM-2629': 18.692537220228665, 'CACM-1506': 18.676860226756293, 'CACM-1938': 18.576999215150295, 'CACM-2632': 18.362258249566853, 'CACM-1544': 18.342545507211913, 'CACM-3127': 18.32191103451082, 'CACM-1161': 18.11686703007412, 'CACM-2371': 17.999816661076768, 'CACM-1680': 17.811346573313667, 'CACM-2740': 17.74654363182257, 'CACM-3025': 17.651602617308413, 'CACM-3048': 17.61615480134931, 'CACM-2357': 17.58227873180795, 'CACM-1647': 17.50741771743242, 'CACM-1464': 17.486469400671375, 'CACM-1168': 17.4790245042065, 'CACM-1264': 17.437050125924394, 'CACM-0585': 17.24829454655485, 'CACM-1750': 17.033853569824924, 'CACM-2541': 16.9225530

KeyError: 'udo'

In [157]:
expanded_query_list = []

In [158]:
def pseudo_relavance(query,bm25_score_dict):
    k = 10
    #Generate query vector
    query_vector = {}
    query_list = query.split(" ")
    
    for query_word in query_list:
        if(query_vector.get(query_word)):
            query_vector[query_word] += 1
        else:
            query_vector[query_word] = 1
            
    for key,val in unigram_invertedlist_count.items():
        if not query_vector.get(key):
            query_vector[key] = 0
    relevance_vector,relevance_vector_magnitude = generate_relevance_vector(query,bm25_score_dict,k)
    
    non_relevance_vector,non_relevance_vector_magnitude = generate_non_relevance_vector(query,bm25_score_dict,k)

    expanded_q = query_expansion(query,query_vector,relevance_vector,relevance_vector_magnitude,
                    non_relevance_vector,non_relevance_vector_magnitude)
   
    return(expanded_q)

In [159]:
def generate_relevance_vector(query,bm25_score_dict,k):
    sorted_doc = dict(sorted(bm25_score_dict.items(), key=operator.itemgetter(1), reverse=True)[:k])
    relevance_vector = {}
    for key,val in sorted_doc.items():
        with open ('corpus/'+key+'.txt', 'r') as f: 
            doc_list = f.read().split()
            for term in doc_list:
                if(relevance_vector.get(term)):
                    relevance_vector[term] += 1
                else:
                    relevance_vector[term] = 1

            for token,val in unigram_invertedlist_count.items():
                if not relevance_vector.get(token):
                        relevance_vector[token] = 0
                        
    relevance_vector_magnitude = 0
    for key,val in relevance_vector.items():
        relevance_vector_magnitude += float(val**2)
    
    relevance_vector_magnitude = math.sqrt(relevance_vector_magnitude)
    return(relevance_vector,relevance_vector_magnitude)

In [160]:
def generate_non_relevance_vector(query,bm25_score_dict,k):
    sorted_doc = dict(sorted(bm25_score_dict.items(), key=operator.itemgetter(1), reverse=True)[k:])
    non_relevance_vector = {}
    for key,val in sorted_doc.items():
        with open ('corpus/'+key+'.txt', 'r') as f: 
            doc_list = f.read().split()
            for term in doc_list:
                if(non_relevance_vector.get(term)):
                    non_relevance_vector[term] += 1
                else:
                    non_relevance_vector[term] = 1

            for token,val in unigram_invertedlist_count.items():
                if not non_relevance_vector.get(token):
                        non_relevance_vector[token] = 0
                        
    non_relevance_vector_magnitude = 0
    for key,val in non_relevance_vector.items():
        non_relevance_vector_magnitude += float(val**2)
    
    non_relevance_vector_magnitude = math.sqrt(non_relevance_vector_magnitude)
    return(non_relevance_vector,non_relevance_vector_magnitude)

In [161]:
def query_expansion(query,query_vector,relevance_vector,relevance_vector_magnitude,\
                    non_relevance_vector,non_relevance_vector_magnitude):
    query_expansion_dict = {}
    for term,val in unigram_invertedlist_count.items():
        query_expansion_dict[term] = query_vector[term]+ (0.5/relevance_vector_magnitude) * relevance_vector[term] -\
        (0.15/non_relevance_vector_magnitude) * non_relevance_vector[term]
    
    query_expansion_dict = dict(sorted(query_expansion_dict.items(), key=operator.itemgetter(1), reverse=True))
    
    expanded_query = query
    no_extra_query_terms = 15
    for i in range(no_extra_query_terms):
        term =  list(query_expansion_dict.keys())[i]
        if term not in query:
            expanded_query+= " "+term
    return expanded_query

In [163]:
def get_all_new_queries():
    for old_query in range(len(all_queries)):
        expanded_q = pseudo_relavance(all_queries[old_query],bm25_score_dict)
        expanded_query_list.append(expanded_q)
        
    return expanded_query_list

all_expanded_queries = get_all_new_queries()

for i in range(len(all_expanded_queries)):
    bm25_score_dict_expanded = populate_bm25(all_expanded_queries[i])
    print(bm25_score_dict_expanded)

{'CACM-1519': 24.64291001264796, 'CACM-1605': 23.148162233549694, 'CACM-1410': 22.33330316789089, 'CACM-1591': 22.29077525815539, 'CACM-1033': 22.288959392145333, 'CACM-1506': 21.989576069246546, 'CACM-1161': 21.101968184738794, 'CACM-2319': 20.927837849632983, 'CACM-0585': 20.79347097377166, 'CACM-2379': 20.548433949383885, 'CACM-1698': 20.509085403312735, 'CACM-1680': 20.394312018199077, 'CACM-1938': 20.27476212484826, 'CACM-2054': 20.156604992450905, 'CACM-3048': 20.112135059644178, 'CACM-1844': 19.64595985147516, 'CACM-1264': 19.588225124500664, 'CACM-1750': 19.57764677467663, 'CACM-1523': 19.44969520614981, 'CACM-3127': 19.300844835949388, 'CACM-1544': 19.188042794803522, 'CACM-3025': 19.088333428950925, 'CACM-2371': 19.062308896129696, 'CACM-2357': 19.002872646227104, 'CACM-2380': 18.960415721057554, 'CACM-2947': 18.8593754495853, 'CACM-1168': 18.649673825895533, 'CACM-1315': 18.431842043017898, 'CACM-2535': 18.31534295256021, 'CACM-1827': 18.041805239516613, 'CACM-0637': 17.8308

KeyError: 'udo'

In [150]:
unigram_termcount['CACM-2246']

127